In [1]:
import quasar
import numpy as np

In [2]:
def random_so4():
    
    params = 2.0* np.pi * np.random.randn(6)
    gate = quasar.Gate.SO4(
        A=params[0],
        B=params[1],
        C=params[2],
        D=params[3],
        E=params[4],
        F=params[5],
        )
    return gate

In [3]:
def random_statevector():
    
    return random_so4().U[:,0]

In [4]:
psi = random_statevector()
psi = np.zeros_like(psi)
psi[0] = 1.0
U = random_so4()
V = random_so4()
U2 = U.U
V2 = V.U

In [5]:
I, X, Y, Z = quasar.Pauli.IXYZ()
O = 0.1 * I 
O += 0.2 * X[0] + 0.3 * Y[0] + 0.4 * Z[0]
O += 0.5 * X[1] + 0.6 * Y[1] + 0.7 * Z[1]
O2 = O.compute_hilbert_matrix()
I2 = np.eye(4, dtype=np.complex128)

In [6]:
psiU = np.dot(U2, psi)
psiV = np.dot(V2, psi)
O3 = np.dot(psiU.conj(), np.dot(O2, psiV))
I3 = np.dot(psiU.conj(), np.dot(I2, psiV))
print(O3)
print(I3)

(-0.22274429164632795-0.4220961739333571j)
(0.6171391106244227+0j)


In [7]:
circuit = quasar.Circuit(N=3)
circuit.H(2)
circuit.X(2)
circuit.add_gate(quasar.ControlledGate(U), (2, 0, 1))
circuit.X(2)
circuit.add_gate(quasar.ControlledGate(V), (2, 0, 1))
print(circuit)

T  : |0|1|2   |3|4   |
                      
q0 : -----SO4A---SO4A-
          |      |    
q1 : -----SO4B---SO4B-
          |      |    
q2 : -H-X-@----X-@----

T  : |0|1|2   |3|4   |


In [8]:
I4R = X[2]
I4I = Y[2]
backend = quasar.QuasarSimulatorBackend()
I5R = quasar.run_observable_expectation_value(backend, circuit, pauli=I4R)
I5I = quasar.run_observable_expectation_value(backend, circuit, pauli=I4I)
I5 = I5R + 1.j * I5I
print(I5)
print(I3)

(0.6171391106244226+0j)
(0.6171391106244227+0j)


In [9]:
O4R = X[2] * O
O4I = Y[2] * O
backend = quasar.QuasarSimulatorBackend()
O5R = quasar.run_observable_expectation_value(backend, circuit, pauli=O4R)
O5I = quasar.run_observable_expectation_value(backend, circuit, pauli=O4I)
O5 = O5R + 1.j * O5I
print(O5)
print(O3)

(-0.2227442916463278-0.42209617393335697j)
(-0.22274429164632795-0.4220961739333571j)
